<h1>Table des matières<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Sujet-de-TP-:-modèle-de-langage-avec-un-RNN-&quot;Vanilla&quot;" data-toc-modified-id="Sujet-de-TP-:-modèle-de-langage-avec-un-RNN-&quot;Vanilla&quot;-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Sujet de TP : modèle de langage avec un RNN "Vanilla"</a></span></li><li><span><a href="#Préparation-des-données" data-toc-modified-id="Préparation-des-données-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Préparation des données</a></span></li><li><span><a href="#imports-et-accès-à-un-GPU" data-toc-modified-id="imports-et-accès-à-un-GPU-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>imports et accès à un GPU</a></span></li><li><span><a href="#Chargement-du-corpus" data-toc-modified-id="Chargement-du-corpus-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Chargement du corpus</a></span></li><li><span><a href="#Créer-la-classe-du-modèle" data-toc-modified-id="Créer-la-classe-du-modèle-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Créer la classe du modèle</a></span></li><li><span><a href="#Instancier-le-modèle" data-toc-modified-id="Instancier-le-modèle-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Instancier le modèle</a></span></li><li><span><a href="#Apprentissage" data-toc-modified-id="Apprentissage-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Apprentissage</a></span></li><li><span><a href="#Tester-le-modèle-sur-des-phrases" data-toc-modified-id="Tester-le-modèle-sur-des-phrases-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Tester le modèle sur des phrases</a></span></li></ul></div>

# Sujet de TP : modèle de langage avec un RNN "Vanilla"

Dans ce TP, nous allons créer un RNN simple (un "Vanilla RNN") qui prédit le mot qui suit un début de phrase. Il s'agit d'un modèle de langage.

Nous allons entraîner ce RNN sur un tout petit sous-ensemble de textes provenant du corpus Librivox French (https://librivox.org) qui regroupe des audiobooks.

Une fois entraîné, vous pouvez compléter un début de phrase en faisant des prédictions avec le modèle.

Nous utilisons les balises < unk > et < eos > pour les mots qui ne sont pas dans notre vocabulaire et pour indiquer une fin de phrase, respectivement.
    
Nous avons restreint le vocabulaire à la liste de mots apparaissant au moins 4 fois dans Librivox.

# imports et accès à un GPU

In [1]:
# # Pour Google Colab
# import sys, os
# if 'google.colab' in sys.modules:
#     from google.colab import drive
#     drive.mount('/content/gdrive')
#     file_name = 'vrnn_demo_librivox_fr.ipynb'
#     import subprocess
#     path_to_file = subprocess.check_output('find . -type f -name ' + str(file_name), shell=True).decode("utf-8")
#     print(path_to_file)
#     path_to_file = path_to_file.replace(file_name,"").replace('\n',"")
#     os.chdir(path_to_file)
#     !pwd

In [2]:
import os
import torch
import torch.nn.functional as F
import torch.nn as nn

import math
import time
import re

Avec ou sans GPU ?

Il est recommandé d'exécuter ce code sur GPU :<br> 
* Temps pour 1 epoch sur CPU : 153 sec ( 2.55 min)<br> 
* Temps pour 1 epoch sur GPU : 8.4 sec avec une GeForce GTX 1080 Ti <br>

In [3]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [4]:
#device= torch.device("cuda")
device= torch.device("cpu")
print(device) #, torch.cuda.get_device_name(device=None))

cpu


In [5]:
torch.cuda.empty_cache()

# Préparation des données

Nous vous fournissons les données précalculées ci-dessous: chaque mot est associé à un identifiant unique, et les phrases sont alors stockées comme des vecteurs d'entiers

# Chargement du corpus


Téléchargement des données

In [6]:
!wget -O dataset.zip https://www.irit.fr/~Thomas.Pellegrini/ens/RNN/librivox_fr_PTfiles.zip
!ls -alth dataset.zip
!unzip -qq dataset.zip 

--2024-04-05 10:52:18--  https://www.irit.fr/~Thomas.Pellegrini/ens/RNN/librivox_fr_PTfiles.zip
Resolving www.irit.fr (www.irit.fr)... 141.115.28.2
Connecting to www.irit.fr (www.irit.fr)|141.115.28.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 946446 (924K) [application/zip]
Saving to: ‘dataset.zip’

dataset.zip         100%[===================>] 924,26K  5,69MB/s    in 0,2s    

2024-04-05 10:52:18 (5,69 MB/s) - ‘dataset.zip’ saved [946446/946446]

-rw-rw-r-- 1 ozgur ozgur 925K déc.   2  2019 dataset.zip


Vérifier que vous avez bien généré les données : 

In [7]:
path_data='./'

flag_train_data = os.path.isfile(path_data + 'librivox_fr/train_data.pt') 
flag_test_data = os.path.isfile(path_data + 'librivox_fr/test_data.pt') 

flag_idx2word = os.path.isfile(path_data + 'librivox_fr/idx2word.pt') 
flag_word2idx = os.path.isfile(path_data + 'librivox_fr/word2idx.pt') 

if flag_idx2word==False or flag_test_data==False or flag_train_data==False or flag_word2idx==False:
    print('Librivox_fr dataset manquant')


Charger train_data et test_data et afficher les dimensions des deux tensors

In [8]:
train_data  =  torch.load(path_data+'librivox_fr/train_data.pt')
test_data   =  torch.load(path_data+'librivox_fr/test_data.pt')

# TODO : afficher les dimensions des deux tensors
print("train_data:", train_data.size())
print("test_data:", test_data.size())

train_data: torch.Size([20542, 20])
test_data: torch.Size([74, 20])


Charger les dictionnaires idx2word et word2idx :

In [9]:
word2idx  =  torch.load(path_data + 'librivox_fr/word2idx.pt')
idx2word  =  torch.load(path_data + 'librivox_fr/idx2word.pt')

print(word2idx)
print(idx2word)

{'<unk>': 0, 'enfant': 1, 'si': 2, 'j’étais': 3, 'roi': 4, 'je': 5, 'donnerais': 6, 'l’empire': 7, 'et': 8, 'mon': 9, 'char': 10, 'sceptre': 11, 'peuple': 12, 'à': 13, 'genoux': 14, 'ma': 15, 'couronne': 16, 'd’or': 17, 'mes': 18, 'bains': 19, 'de': 20, 'porphyre': 21, 'qui': 22, 'la': 23, 'mer': 24, 'ne': 25, 'peut': 26, 'suffire': 27, '<eos>': 28, 'pour': 29, 'un': 30, 'regard': 31, 'vous': 32, 'dieu': 33, 'terre': 34, 'l’air': 35, 'avec': 36, 'les': 37, 'ondes': 38, 'anges': 39, 'démons': 40, 'devant': 41, 'loi': 42, 'le': 43, 'profond': 44, 'chaos': 45, 'aux': 46, 'entrailles': 47, 'l’éternité': 48, 'l’espace': 49, 'cieux': 50, 'mondes': 51, 'baiser': 52, 'toi': 53, 'c’est': 54, 'fatigue': 55, 'amoureuse': 56, 'tous': 57, 'des': 58, 'bois': 59, 'parmi': 60, 'vers': 61, 'grises': 62, 'choeur': 63, 'petites': 64, 'voix': 65, 'ô': 66, 'frêle': 67, 'frais': 68, 'murmure': 69, 'cela': 70, 'ressemble': 71, 'au': 72, 'cri': 73, 'doux': 74, 'que': 75, 'l’herbe': 76, 'tu': 77, 'dirais': 78,

La première phrase du texte train_librivox_fr_50words_max_15200.txt est : 

"enfant si j’étais roi je donnerais l’empire et mon char et mon sceptre et mon peuple à genoux et ma couronne d’or et mes bains de porphyre et mes flottes à qui la mer ne peut suffire"

**QUESTIONS**

1.   Comment est-elle stockée dans le tenseur train_data ?

    Afficher les 38 premiers mots de train_data, avec leur identifiant entier, qui correspondent à cette phrase jusqu'à < eos >. Vous devez obtenir un affichage similaire à : 

    1:enfant 2:si 3:j’étais 4:roi 5:je 6:donnerais 7:l’empire 8:et 9:mon 10:char 8:et 9:mon 11:sceptre 8:et 9:mon 12:peuple 13:à 14:genoux 8:et 15:ma 16:couronne 17:d’or 8:et 18:mes 19:bains 20:de 21:porphyre 8:et 18:mes 0:<unk> 13:à 22:qui 23:la 24:mer 25:ne 26:peut 27:suffire 28:<eos> 

2.   Où est stockée la deuxième phrase ?


In [52]:
## TODO
# chaque phrase est stockée sous forme d'une séquence d'indices correspondant aux mots de la phrase.
# Les mots sont représentés par leur indice dans le vocabulaire
for i in range(38):
    indice = train_data[i,0].item()
    print(f'{indice}:{idx2word[indice]}', end=' ')

# 2
# la deuxième phrase est stockée à partir de l'indice après la balise <eos>. 
# chaque phrase est séparée dans le tenseur par la balise <eos> qui indique la fin d'une phrase.


1:enfant 2:si 3:j’étais 4:roi 5:je 6:donnerais 7:l’empire 8:et 9:mon 10:char 8:et 9:mon 11:sceptre 8:et 9:mon 12:peuple 13:à 14:genoux 8:et 15:ma 16:couronne 17:d’or 8:et 18:mes 19:bains 20:de 21:porphyre 8:et 18:mes 0:<unk> 13:à 22:qui 23:la 24:mer 25:ne 26:peut 27:suffire 28:<eos> 

**En déduire comment les données sont présentées au futur modèle.**


In [53]:
max(torch.unique(train_data)), len((torch.unique(train_data)))

(tensor(9573), 9574)

In [44]:
max(torch.unique(test_data)), len((torch.unique(test_data)))

(tensor(9575), 503)

Notez qu'il y a un problème avec les données de test : lequel ? Comment pourrait-on le réparer

Quelques constantes associées au dataset

In [54]:
bs = 20 # taille des batches : fixée ici dans le prétraitement
seq_length = 35 # taille des "phrases" à donner au réseau

vocab_size = 9574 

# Créer la classe du modèle

1.   Compléter la définition du modèle à trois couches suivant, en indiquant les bonnes dimensions.


2.   Compléter la définition de la fonction forward.

In [55]:
class three_layer_recurrent_net(nn.Module):

    def __init__(self, hidden_size, vocab_size):
        
        super(three_layer_recurrent_net, self).__init__()
        
        self.layer1 = nn.Embedding( vocab_size  , hidden_size  )
        self.layer2 = nn.GRU( hidden_size , hidden_size, num_layers = 1  )
        self.layer3 = nn.Linear( hidden_size, vocab_size )

        
    def forward(self, word_seq, h_init ):
        
        g_seq               =   self.layer1(word_seq)  # Transformation des mots en vecteurs denses
        h_seq , h_final     =   self.layer2(g_seq, h_init)
        score_seq           =   self.layer3(h_seq)
        
        return score_seq,  h_final 


# Instancier le modèle

Il est possible d'accéder aux paramètres d'un modèle à l'aide de la méthode net.parameters().


Compléter la fonction display_num_param qui affiche le nombre de paramètres d'un réseau donné en argument (net).

In [58]:
def display_num_param(net):
    num_param = sum(p.numel() for p in net.parameters() if p.requires_grad)
    print(f"Le nombre de paramètres: {num_param}")

Instancier le réseau dans une variable appelée net. Le réseau doit avoir 150 neurones pour la couche récurrente. Combien de paramètres au total contient le modèle?

In [65]:
# TODO
hidden_size = 150
net = three_layer_recurrent_net(hidden_size=hidden_size, vocab_size=vocab_size)
display_num_param(net)

Le nombre de paramètres: 3017674


Envoyer le modèle sur le GPU (si vous utilisez un gpu)

In [66]:
net = net.to(device)

Initialiser les poids de la couche embedding et de la couche linéaire avec une distribution uniforme sur [-0.1, 0.1]

In [67]:
net.layer1.weight.data.uniform_(-0.1, 0.1)

net.layer3.weight.data.uniform_(-0.1, 0.1)

print('')

# Apprentissage

Définir la fonction de coût entropie croisée et les hyperparamètres suivants : 
* learning rate initial : my_lr=1
* taille des séquences : seq_length=35

In [68]:
criterion = nn.CrossEntropyLoss()

my_lr = 1

seq_length = 35

Lors de l'apprentissage, pour éviter le phénomène d'explosion du gradient, nous allons utiliser une fonction qui normalise les valeurs du gradient

In [69]:
def normalize_gradient(net):

    grad_norm_sq=0

    for p in net.parameters():
        grad_norm_sq += p.grad.data.norm()**2

    grad_norm=math.sqrt(grad_norm_sq)
   
    if grad_norm<1e-4:
        net.zero_grad()
        print('norme du gradient proche de zéro')
    else:
        for p in net.parameters():
             p.grad.data.div_(grad_norm)

    return grad_norm



Voici une fonction qui évaluerait le réseau sur le jeu de test (non-utilisée à cause du problème dans les données de test)

In [64]:
# def eval_on_test_set():

#     running_loss=0
#     num_batches=0    
       
#     h = torch.zeros(1, bs, hidden_size)
    
#     h=h.to(device)

#     for count in range( 0 , 74-seq_length ,  seq_length) :
               
#         minibatch_data =  test_data[ count   : count+seq_length   ]
#         minibatch_label = test_data[ count+1 : count+seq_length+1 ]
        
#         minibatch_data=minibatch_data.to(device)
#         minibatch_label=minibatch_label.to(device)
                                  
#         scores, h  = net( minibatch_data, h )
        
#         minibatch_label =   minibatch_label.view(  bs*seq_length ) 
#         scores          =            scores.view(  bs*seq_length , vocab_size)
        
#         loss = criterion(  scores ,  minibatch_label )    
        
#         h=h.detach()
            
#         running_loss += loss.item()
#         num_batches += 1        
    
#     total_loss = running_loss/num_batches 
#     print('test: exp(loss) = ', math.exp(total_loss)  )
        

Compléter la boucle d'apprentissage aux endroits indiqués par ?? et entraîner le modèle sur 10 epochs

In [70]:
start=time.time()

for epoch in range(10):
    
    # garder le learning rate à 1.0 pour les 4 premières epochs, puis diviser par 1.1 à chaque epoch
    if epoch >= 4:
        my_lr = my_lr / 1.1
    
    # créer un nouvel optimizer de type SGD et lui passer les paramètres du modèle et le learning rate actualisé.   
    optimizer=torch.optim.SGD( net.parameters() , lr=my_lr)
        
    # initialisation du coût et du nombre de batchs à chaque nouvelle epoch 
    running_loss=0
    num_batches=0    
    
    # initialiser h par un vecteur de zéros avec les bonnes dimensions requises :
    h = torch.zeros( 1, bs, hidden_size)

    # envoi au gpu    
    h=h.to(device)
    
    for count in range( 0 , 20542-seq_length ,  seq_length):
             
        # Mettre les gradients à zéro
        optimizer.zero_grad()
        
        # créer un minibatch
        minibatch_data =  train_data[ count   : count+seq_length   ]
        minibatch_label = train_data[ count+1 : count+seq_length+1 ]        
        
        # envoi au gpu
        minibatch_data=minibatch_data.to(device)
        minibatch_label=minibatch_label.to(device)
        
        # Detacher h pour ne pas backpropager sur toutes les séquences depuis le début de l'epoch
        h=h.detach()
        # Dire à Pytorch de tracker les opérations sur h pour le minibatch courant
        h=h.requires_grad_()
                       
        # Réaliser une Passe forward
        scores, h  = net( minibatch_data, h )
        
        # Reshape les tenseurs scores et labels pour obtenir une longueur de bs*seq_length
        scores          =            scores.view(  bs*seq_length , vocab_size)  
        minibatch_label =   minibatch_label.view(  bs*seq_length )       
        
        # Calculer la loss moyenne
        loss = criterion(  scores ,  minibatch_label)
        
        # Passe backward pour calculer les gradients dL/dR, dL/dV et dL/dW
        loss.backward()

        # Normaliser les gradients et faire une itération de SGD : R=R-lr(dL/dR), V=V-lr(dL/dV), ...
        normalize_gradient(net)
        optimizer.step()
        
        # Actualiser le coût par epoch et le nb de batches traités  
        running_loss += loss.item()
        num_batches += 1
        
        
    # calcul du coût sur tout le training set
    total_loss = running_loss / num_batches
    elapsed = time.time()-start
    
    print('')
    print('epoch=',epoch, '\t time=', elapsed,'\t lr=', my_lr, '\t exp(loss)=',  math.exp(total_loss))
    
    # Estimer la performance sur le jeu de test (bug pour l'instant) 
    #     eval_on_test_set() 



epoch= 0 	 time= 33.359201192855835 	 lr= 1 	 exp(loss)= 654.997941270097

epoch= 1 	 time= 69.49244475364685 	 lr= 1 	 exp(loss)= 332.6747800106261

epoch= 2 	 time= 106.08562326431274 	 lr= 1 	 exp(loss)= 239.53595064744744

epoch= 3 	 time= 142.7331244945526 	 lr= 1 	 exp(loss)= 197.0229700457691

epoch= 4 	 time= 179.5727994441986 	 lr= 0.9090909090909091 	 exp(loss)= 169.83175632491808

epoch= 5 	 time= 210.05920577049255 	 lr= 0.8264462809917354 	 exp(loss)= 151.5688243850608

epoch= 6 	 time= 243.04313039779663 	 lr= 0.7513148009015777 	 exp(loss)= 138.19846415228136

epoch= 7 	 time= 275.9627137184143 	 lr= 0.6830134553650705 	 exp(loss)= 127.91137227758615

epoch= 8 	 time= 306.4337558746338 	 lr= 0.6209213230591549 	 exp(loss)= 119.73966678391342

epoch= 9 	 time= 336.5817458629608 	 lr= 0.5644739300537771 	 exp(loss)= 113.07566363381909


**Question** : quelle est la taille des tenseurs suivants ?

* minibatch_data, minibatch_label,
* h et scores avant le reshape
* h, scores après le reshape


In [72]:
#  minibatch_label -> (bs*seq_length)
# scores          -> (bs*seq_length , vocab_size)
# minibatch_data -> (seq_length, bs)
minibatch_data.size(), minibatch_label.size(), scores.size()

(torch.Size([35, 20]), torch.Size([700]), torch.Size([700, 9574]))

# Tester le modèle sur des phrases

Voici une fonction qui prend une phrase et qui la convertit en tenseur exploitable pour le réseau

In [73]:
def sentence2vector_librivox_fr(sentence):
    words = sentence.split()
    x = torch.LongTensor(len(words),1)
    for idx, word in enumerate(words):
        word = re.sub("'", "_", word)
        if word not in word2idx:
            print('Vous avez entrer un mot hors-vocabulaire.')
            print('--> Enlever lettres majuscules et ponctuation')
            print("mot --> <unk> avec index 0")
            x[idx,0]=0            
        else:
            x[idx,0]=word2idx[word]
    return x


In [74]:
sentence1 = "on entendait vaguement au dehors les"

sentence2 = "hier je luttai de la sorte contre le grand"

sentence3 = "il connaissait la route et nous avons"

# ou bien créer votre propre phrase. Il ne fauit pas utiliser de majuscules ni de ponctuation.
# Chaque mot doit être dans le lexique.
sentence4= "il est beaucoup"

# Choisir le phrase ici : 
mysentence = sentence1

Convertir la phrase choisie et envoi au GPU

In [75]:
minibatch_data=sentence2vector_librivox_fr(mysentence)
      
minibatch_data=minibatch_data.to(device)

print(minibatch_data)

tensor([[ 366],
        [1383],
        [1884],
        [  72],
        [ 127],
        [  37]])


Définir un hidden state initial à zero, et exécuter le RNN sur la phrase

In [76]:
h = torch.zeros(1, 1, hidden_size)
h=h.to(device)

scores , h = net( minibatch_data , h )

Écrire une fonction show_next_word qui prend en entrée scores et qui affiche les 30 mots les plus probables prédits par le réseau, en indiquant leur probabilité par un pourcentage.

L'affichage doit ressembler à : 

```
on entendait vaguement au dehors les ... 

6.0%	 autres
3.2%	 <unk>
2.0%	 hommes
1.9%	 plus
1.6%	 yeux
...
```

Vous utiliserez la fonction torch.topk()
Aide : https://pytorch.org/docs/stable/torch.html?highlight=topk#torch.topk


In [ ]:
def show_next_word(scores):
    pass

Afficher la prédiction du prochain mot par le réseau

In [ ]:
print(mysentence, '... \n')

show_next_word(scores)

Le mot < unk > sera presque toujours le plus probable. 

Voici une fonction get_next_word, variante de show_next_word qui retourne le mot le plus fréquent. 

Si ce mot est < unk >, cette fonction retourne le duexième mot le plus probable.

In [ ]:
def get_next_word(scores):
    prob=F.softmax(scores,dim=2)
    num_word_display = 2
    p=prob[-1].squeeze()
    p, word_idx = torch.topk(p, num_word_display)
#     print(p, word_idx)
    if word_idx[0] == 0:
        return idx2word[word_idx[1]]
    else:
        return idx2word[word_idx[0]]
    

In [ ]:
next_word = get_next_word(scores)

Écrire un bout de code qui prédit une phrase entière à partir de mysentence.

Cette phrase sera considérée comme terminée sur le modèle prédit < eos > ou bien après 10 itérations maximum.

In [ ]:
mysentence = sentence1
print(mysentence + '...')

i= 0
not_finished = True
while ...
